# 概要
こんな人向け：機械学習の勉強を始めたばかりの人。何をしたらいいかわからない人。ノートのコピペだけで終わっちゃう人。

データの内容からモデル作成までを解説します。

# 1. データ確認

In [ ]:
import numpy as np
import pandas as pd

numpy：各種計算に使う。

pandas：表計算やデータのロードに使う。

これら２つはほぼ確実に利用するのでとりあえずインポートしてもいいくらいです。

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
print(train.shape)
train.head()

pandasの.read_csvでCSVファイルを読み込みます。

学習用データのサイズは約300万行6列です。

.headで最初の５行を表示してくれます。

今回予測するのは商品の販売数を示す"item_cnt_day"の、shopごと、かつitemごとの、翌月の総数です。（マイナスは入荷？）

In [ ]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test.head()

testは実際に予測したいデータです。当然ですが"item_cnt_day"はありません。

In [ ]:
sample = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
sample.head()

提出形式を示すデータです。このファイルを直接submitしてもスコアが出ます。

"ID"と"item_cnt_month"の２列が必要ですね。

"month"になっているので"item_cnt_day"を月の販売総数に変換しないといけません。

# 2. date

In [ ]:
train.head()

In [ ]:
train.info()

"date"は日付ですが文字列になっているので使いにくいです。

新しく年と月と日の列を作りましょう。

In [ ]:
train["date"][0]

こんな感じで文字になっています。

In [ ]:
train["date"][0][:2]

[:数字]で指定した数字までの文字を切り取れます。

In [ ]:
train["date"].apply(lambda x: x[:2])

.applyで各データに特定の処理を実行できます。さっきの文字の切り取りを渡すと全行の２文字目までを切り取ってくれます。

In [ ]:
train["day"] = train["date"].apply(lambda x: int(x[:2]))
train.head()

日を示す"day"として切り取っておきました。intに入れると文字を整数にします。

In [ ]:
train["day"].describe()

.describeで統計値が確認できます。最小が１で最大が31なので確かに日ですね。

In [ ]:
train["date"][0][3:5]

今度は月をとりだしましょう。[数値：数値]で間の文字をとれます。

In [ ]:
train["month"] = train["date"].apply(lambda x: int(x[3:5]))
train["month"].describe()

月を"month"としました。たしかに１～12の数値です。

In [ ]:
train["year"] = train["date"].apply(lambda x: int(x[6:]))
train["year"].describe()

年も同じ。

In [ ]:
train.head()

これで年月日を各列に出力できました。

In [ ]:
train.drop(columns = "date", inplace = True)
train.head()

"date"はもう不要なので消しておきました。

In [ ]:
print(train["year"].max())

最も新しい年は2015年です。

In [ ]:
train_2015 = train.loc[train["year"] == 2015]
print(train_2015.shape)

.locで2015年のデータを取り出しました。60万行あります。

In [ ]:
print(train_2015["month"].max())

In [ ]:
print(train_2015.loc[train_2015["month"] == 10]["day"].max())

2015年の中でも10月31日が最後みたいです。

今回のタスクは翌月つまり2015年11月の各ショップ各商品の販売総数を予測しろってことですね。

In [ ]:
test.head()

testには年月日がありません。

月の総数を予測するので日は不要ですが、年とは足しておきましょう。

In [ ]:
test["year"] = 2015
test["month"] = 11
test.head()

# 3. date_block_num

In [ ]:
train["date_block_num"].describe()

"date_block_num"という列がありますが、これは2013年１月を０として2013年２月が１...と増えていく値です。

In [ ]:
train.loc[train["date_block_num"] == 0]

In [ ]:
train.loc[train["date_block_num"] == 1]

In [ ]:
train.loc[train["date_block_num"] == 33]

In [ ]:
test["date_block_num"] = 34
test.head()

なので予測したい2015年11月は34です。

# 4. all_data

In [ ]:
train.loc[(train["shop_id"] == 0) & (train["item_id"] == 32)]

例えば店舗０の商品30を見てみます。

2013年１月と２月に売れていますがそれ以降は全く売れていません。

In [ ]:
train.loc[(train["shop_id"] == 59) & (train["item_id"] == 22162)]

一方で店舗59の商品22162は2015年２月以降にしか売れていません。

上記表は言い換えると2015年６,７月での売り上げが０ということです。

このようにtrainデータのままでは歯抜けになっているので、６,７月の売上は０という行も作りましょう。

In [ ]:
shops = train["shop_id"].unique()
items = train["item_id"].unique()

全年月における全店舗の全商品のデータを作ります。

.uniqueで全ての水準を取得できます。

In [ ]:
from tqdm.notebook import trange
all_data = []
for i in trange(34):
    for shop in shops:
        for item in items:
            all_data.append([i, shop, item])
    
all_data = pd.DataFrame(all_data, columns = ["date_block_num", "shop_id", "item_id"])

学習データの"date_block_num"は０～33です。

全ての"shop_id"は.uniqueで取得しました。"item_id"も同じです。

これらをリストにして行方向に次々と積み上げていきましょう。

trangeはプログレスバーを作るコードです。

実行するとall_dataとして全年月日における全店舗の全商品が網羅されたデータが作られます。

In [ ]:
print(all_data.shape)
print(train.shape)

trainよりサイズが大きくなっています。

# 5. item_cnt_month
月の売上総数を計算します。

In [ ]:
train.groupby(["date_block_num", "shop_id", "item_id"], as_index = False)["item_cnt_day"].sum()

.groupbyでカテゴリごとに集計できます。

"date_block_num"ごとの"shop_id"ごとの"item_id"ごとの販売総数を計算しました。

In [ ]:
train.loc[(train["date_block_num"] == 0) & (train["shop_id"] == 0) & (train["item_id"] == 32)]

確かに合計数があっています。

In [ ]:
all_data = all_data.merge(
    train.groupby(["date_block_num", "shop_id", "item_id"], as_index = False)["item_cnt_day"].sum(),
    on = ["date_block_num", "shop_id", "item_id"],
    how = "left"
)
all_data.head()

pandasの.mergeでデータを結合しました。

onはどの列を参照して結合するか指定します。howを"left"にすると元のall_dataに存在しないカテゴリは無視されます。

NaNになっている部分は販売数が０です。

In [ ]:
all_data.rename(columns = {"item_cnt_day" : "item_cnt_month"}, inplace = True)
all_data.fillna(0, inplace = True)
all_data.head()

列名を変更し販売がない行に０を入れました。

In [ ]:
all_data["year"] = 0
all_data["month"] = 0
year = 2013
month = 1
for date_block_num in range(34):
    all_data.loc[all_data["date_block_num"] == date_block_num, "year"] = year
    all_data.loc[all_data["date_block_num"] == date_block_num, "month"] = month
    month += 1
    if month == 13:
        year += 1
        month = 1

年月がなくなったので新しく作成します。

In [ ]:
all_data.head()

In [ ]:
all_data.tail()

# 6. モデル作成
モデルはLightGBMを使います。

他にも初心者向けのモデルがありますが作る手間はほとんど変わらないのでLightGBMにしましょう。

In [ ]:
all_data.head()

In [ ]:
test.head()

all_dataとtestの列を確認します。"ID"は使わないので他の列があればOK。

In [ ]:
import gc
train = all_data.loc[all_data["date_block_num"] != 33]
valid = all_data.loc[all_data["date_block_num"] == 33]

del all_data; gc.collect()

学習データと評価データに分けました。

分割するとall_dataは不要なのでdelで消してgc.collectでメモリを整理しましょう。

In [ ]:
import lightgbm as lgb

lightgbmをインポートしました。

In [ ]:
params = {
    "objective" : "regression",
    "metric" : "rmse",
    "verbosity" : -1
}

学習パラメータを設定します。

objective：モデルの種類。regressionは回帰です。

metric：評価指標。rmseは平均二乗誤差の平方根です。

verbosity：出力形態。設定しなくてもOK。

In [ ]:
use_cols = ["date_block_num", "shop_id", "item_id", "year", "month"]
target = "item_cnt_month"

X_train = train[use_cols]
y_train = train[target]
X_valid = valid[use_cols]
y_valid = valid[target]

学習に使う列名をリスト化しました。

Xは学習に使う列でyは予測したい列です。

In [ ]:
train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)
del train, valid; gc.collect()

.DatasetでLightGBM専用の形式に変換します。

In [ ]:
model = lgb.train(
    params = params, train_set = train_set, valid_sets = [valid_set], num_boost_round = 1000, early_stopping_rounds = 10
)

.trainで学習できます。

params：さっき作ったパラメータ。

train_set：学習データ。

valid_sets：評価データ。

num_boost_round：学習回数。

early_stopping_rounds：何回性能が同じ(頭打ち)だったら学習を止めるか。

# 7. 提出

In [ ]:
X_test = test[use_cols]
preds = model.predict(X_test)

実際に予測したいデータを使って提出データを作ります。

.predictで予測できます。

In [ ]:
submit = pd.DataFrame()
submit["ID"] = test["ID"]
submit[target] = preds
submit.head()

.DataFrameでカラのデータセットを作りました。

"ID"はtestのものをそのまま使いましょう。

"item_cnt_month"はさっき予測したデータです。

In [ ]:
print(submit.shape, sample.shape)

一応サンプルと同じサイズか確認しました。

In [ ]:
submit.to_csv("submission.csv", index = False)

.to_csvでCSVファイルとして出力できます。indexをFalseにしないと余計な列ができます。

右上の"Save Version"で保存し、プレビュー画面下にある"output"から"submit"を押して提出しましょう。